In [33]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/run3jun21no1.txt','a') # xJ polynomials
for n in [0..200]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        print (n,k,time.time()-start)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n+2)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n,rl]] 
    finish = time.time()
    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

In [21]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    data=data+[k-index]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[1, 3/8*x^4 + 1/2*x^2]


In [6]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print poly

3/8*x^4 + 1/2*x^2
69/1024*x^6 - 1/128*x^4 - 3/64*x^2
1/128*x^8 - 29/864*x^6 + 1/216*x^4 + 1/54*x^2
5601/8388608*x^10 - 3821/524288*x^8 + 5195/262144*x^6 - 101/32768*x^4 - 303/32768*x^2
3/65536*x^12 - 16367/18432000*x^10 + 7961/1382400*x^8 - 21809/1728000*x^6 + 373/172800*x^4 + 373/72000*x^2
23003/8589934592*x^14 - 2254159/28991029248*x^12 + 338577733/391378894848*x^10 - 106154827/24461180928*x^8 + 68420351/8153726976*x^6 - 4754693/3057647616*x^4 - 4754693/1528823808*x^2


In [1]:
divisors(12)

[1, 2, 3, 4, 6, 12]

In [8]:
f=x^2+3
print f.coefficients(sparse=True)
print f.coefficients(sparse=False)
print f.coefficients()
print f.list()

[[3, 0], [1, 2]]
[3, 0, 1]
[[3, 0], [1, 2]]
[3, 0, 1]


In [25]:
pf=primeFactorsIn([12,8])
print pf
fpf=flatten(pf)
print fpf
print list(set(fpf))

[[2, 3], [2]]
[2, 3, 2]
[2, 3]


In [27]:
def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

print primeFactorsInList([12,8])


[2, 3]


In [42]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..3]:
    print "--------------------------------------------------------------------"
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
 
    print index;print poly;print cffs;
    dz=dropzeros(cffs)
    print dz
    dens=listdens(dz)
    print dens
    print primeFactorsInList(dens)

--------------------------------------------------------------------
0
x^2
[0, 0, 1]
[1]
[1]
[]
--------------------------------------------------------------------
1
3/8*x^4 + 1/2*x^2
[0, 0, 1/2, 0, 3/8]
[1/2, 3/8]
[2, 8]
[2]
--------------------------------------------------------------------
2
69/1024*x^6 - 1/128*x^4 - 3/64*x^2
[0, 0, -3/64, 0, -1/128, 0, 69/1024]
[-3/64, -1/128, 69/1024]
[64, 128, 1024]
[2]
--------------------------------------------------------------------
3
1/128*x^8 - 29/864*x^6 + 1/216*x^4 + 1/54*x^2
[0, 0, 1/54, 0, 1/216, 0, -29/864, 0, 1/128]
[1/54, 1/216, -29/864, 1/128]
[54, 216, 864, 128]
[2, 3]


In [45]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
for k in [0..10]:
    print "--------------------------------------------------------------------"
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    print index
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    print pf
    lp=len(pf)
    for h in [0..lp-1]:
        if pf[h]>index:no=no+[index]
print "no: ",no
    

--------------------------------------------------------------------
0
[]
--------------------------------------------------------------------
1
[2]
--------------------------------------------------------------------
2
[2]
--------------------------------------------------------------------
3
[2, 3]
--------------------------------------------------------------------
4
[2]
--------------------------------------------------------------------
5
[2, 3, 5]
--------------------------------------------------------------------
6
[2, 3]
--------------------------------------------------------------------
7
[2, 3, 5, 7]
--------------------------------------------------------------------
8
[2, 3, 5]
--------------------------------------------------------------------
9
[2, 3, 5, 7]
--------------------------------------------------------------------
10
[2, 3, 5, 7]
no:  [1]


In [47]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    for h in [0..lp-1]:
        if pf[h]>index:no=no+[index]
print "no: ",no
    

no:  [1]


In [58]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
missing=[]
for k in [1..10]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp<pp:missing=missing=missing+[(index,pp-lp)]
    for h in [0..lp-1]:
        if pf[h]>index:no=no+[index]
print "no: ",no
print missing

no:  [1]
[(4, 1), (6, 1), (8, 1)]


In [83]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
strange=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp<pp:
        print (index,pp-lp)
        strange=strange+[index/2]
print strange


(4, 1)
(6, 1)
(8, 1)
(12, 1)
(14, 1)
(18, 1)
(20, 1)
(24, 1)
(30, 1)
(32, 1)
(38, 1)
(42, 1)
(44, 1)
(48, 1)
(54, 1)
(60, 1)
(62, 1)
(68, 1)
(72, 1)
(74, 1)
(80, 1)
(84, 1)
(90, 1)
(98, 1)
(102, 1)
(104, 1)
(108, 1)
(110, 1)
(114, 1)
(128, 1)
(132, 1)
(138, 1)
(140, 1)
(150, 1)
(152, 1)
(158, 1)
(164, 1)
(168, 1)
(174, 1)
(180, 1)
(182, 1)
(192, 1)
(194, 1)
(198, 1)
(200, 1)
[2, 3, 4, 6, 7, 9, 10, 12, 15, 16, 19, 21, 22, 24, 27, 30, 31, 34, 36, 37, 40, 42, 45, 49, 51, 52, 54, 55, 57, 64, 66, 69, 70, 75, 76, 79, 82, 84, 87, 90, 91, 96, 97, 99, 100]


In [52]:
# See A006254, A111333, and A130291.

3


In [53]:
tuple([2,3])

(2, 3)

In [54]:
ls=[1,3,11,-17]
print #listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    mx=max(tuple(pf))
    if mx>index:no=no+[index]
print "no"
print no
    


no
[1]


In [69]:
for n in [1..5]:
    fn=primes_first_n(n)
    print (fn,(7 in fn))

([2], False)
([2, 3], False)
([2, 3, 5], False)
([2, 3, 5, 7], True)
([2, 3, 5, 7, 11], True)


In [80]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
missing=[]
for k in [1..10]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    pp=prime_pi(index)
    pn=primes_first_n(pp)
    lp=len(pf)
    pp=prime_pi(index)
    if lp<pp:
        lastdenominatorprime=pf[lp-1]
        lastprimelessthanindex=max(tuple(pn))
        print (index,lastdenominatorprime,lastprimelessthanindex)
        



(4, 2, 3)
(6, 3, 5)
(8, 5, 7)


In [81]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
missing=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    pp=prime_pi(index)
    pn=primes_first_n(pp)
    lp=len(pf)
    pp=prime_pi(index)
    if lp<pp:
        lastdenominatorprime=pf[lp-1]
        lastprimelessthanindex=max(tuple(pn))
        print (index,lastdenominatorprime,lastprimelessthanindex)
        



(4, 2, 3)
(6, 3, 5)
(8, 5, 7)
(12, 7, 11)
(14, 7, 13)
(18, 13, 17)
(20, 17, 19)
(24, 19, 23)
(30, 23, 29)
(32, 29, 31)
(38, 31, 37)
(42, 31, 41)
(44, 31, 43)
(48, 31, 47)
(54, 31, 53)
(60, 31, 59)
(62, 31, 61)
(68, 31, 67)
(72, 31, 71)
(74, 31, 73)
(80, 31, 79)
(84, 79, 83)
(90, 83, 89)
(98, 89, 97)
(102, 97, 101)
(104, 101, 103)
(108, 103, 107)
(110, 107, 109)
(114, 109, 113)
(128, 113, 127)
(132, 127, 131)
(138, 127, 137)
(140, 127, 139)
(150, 127, 149)
(152, 127, 151)
(158, 127, 157)
(164, 127, 163)
(168, 127, 167)
(174, 127, 173)
(180, 127, 179)
(182, 127, 181)
(192, 127, 191)
(194, 127, 193)
(198, 127, 197)
(200, 127, 199)
